<a href="https://colab.research.google.com/github/hearts2/Senior-Communication/blob/main/Selenium.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 아래꺼 설치하고나서 기다리기 

In [ ]:
# 설치는 한번만 하면 된다고 함.
!pip install selenium              # 동적 웹페이지를 크롤링하기 위한 라이브러리
!pip install urllib
!pip install pandas
!pip install beautifulsoup4        # 정적 웹페이지를 크롤링을 할 수 있는 라이브러리
!apt install chromium-chromedriver # 크롬드라이버

In [ ]:
import time
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

In [ ]:
chrome_options = webdriver.ChromeOptions()
chrome_options.add_argument('--headless') #내부 창을 띄울 수 없으므로 설정
chrome_options.add_argument('--no-sandbox') # sandbox 기능을 비활성화 하기
chrome_options.add_argument('--disable-dev-shm-usage') # dev/shm/ 폴더를 사용하지 않기
driver = webdriver.Chrome('chromedriver',options=chrome_options) # ,options=chrome_options

In [ ]:
# driver.current_url  = 현재 url을 가르쳐 준다.
# driver.page_source = 현재 페이지 소스를 보여준다.

In [ ]:
# 1페이지 클릭
# driver.find_element(By.XPATH, '//*[@id="pageDiv"]/ul/li[3]/a').click()

In [ ]:
#사용할 URL
driver.get("https://www.myhome.go.kr/hws/portal/cmt/selectCmtSrvCntrListView.do")
html = driver.page_source
bsObject = BeautifulSoup(html, 'html.parser')

In [ ]:
# 해당 페이지 전체 소스코드
temp = bsObject.find_all("table")[0]
temp

<table cellspacing="0" class="bbs_type1" summary="전국민주민센터검색">
<caption>전국민주민센터검색</caption>
<colgroup>
<col width="10%"/>
<col width="35%"/>
<col width="20%"/>
<col width=""/>
</colgroup>
<thead>
<tr>
<th>번호</th>
<th>주민센터명</th>
<th class="al">대표전화번호</th>
<th>주소</th>
</tr>
</thead>
<tbody id="schTbody"><tr><td>1</td><td>청운효자동 주민센터</td><td class="al">02-2148-5002</td><td class="al"><a href="javascript:fnRentalHouseInfoMap('jumin','11','110','10300','청운효자동 주민센터');">서울특별시 종로구 자하문로 92</a></td></tr><tr><td>2</td><td>사직동 주민센터</td><td class="al">02-2148-5033</td><td class="al"><a href="javascript:fnRentalHouseInfoMap('jumin','11','110','11500','사직동 주민센터');">서울특별시 종로구 사직로9길 1</a></td></tr><tr><td>3</td><td>종로1-4가동 주민센터</td><td class="al">02-2148-5243</td><td class="al"><a href="javascript:fnRentalHouseInfoMap('jumin','11','110','13800','종로1-4가동 주민센터');">서울특별시 종로구 종로17길 8</a></td></tr><tr><td>4</td><td>삼청동 주민센터</td><td class="al">02-2148-5062</td><td class="al"><a href="javascript:fnRentalHouseIn

In [ ]:
column = ["번호", "주민센터명", "대표전화번호", "주소"]
df = pd.DataFrame(columns = column)
templen = len(temp.find_all("tr"))
templen
print(df, templen)

Empty DataFrame
Columns: [번호, 주민센터명, 대표전화번호, 주소]
Index: [] 11


In [ ]:
number = ['//*[@id="pageDiv"]/ul/li[3]/a', '//*[@id="pageDiv"]/ul/li[4]/a','//*[@id="pageDiv"]/ul/li[5]/a','//*[@id="pageDiv"]/ul/li[6]/a','//*[@id="pageDiv"]/ul/li[7]/a','//*[@id="pageDiv"]/ul/li[8]/a','//*[@id="pageDiv"]/ul/li[9]/a','//*[@id="pageDiv"]/ul/li[10]/a','//*[@id="pageDiv"]/ul/li[11]/a','//*[@id="pageDiv"]/ul/li[12]/a']
len(number)

10

In [ ]:
# 위에꺼까지 전부 하면?
df4 = pd.DataFrame(columns = column)                    # 빈 데이터프레임

for Z in range(5):
    for k in range(10):                                 # 각 행의 10개를 합침.
        page = driver.find_element(By.XPATH, number[k]) # 페이지 번호위에 마우스를 올려놓음.
        page.send_keys(Keys.ENTER)                      # 페이지 번호 클릭
        time.sleep(1)                                   # 1초간 기다림.
        html = driver.page_source                       # html 가져옴.
        bsObject = BeautifulSoup(html, 'html.parser')   # 해당 페이지 긁어옴.
        temp = bsObject.find_all("table")[0]            # 페이지 중 테이블 가져옴.
        column = ["번호", "주민센터명", "대표전화번호", "주소"]

        df3 = pd.DataFrame(columns = column)            # for 문 돌릴때마다 데이터프레임 초기화
        for i in range(1, 11):
            tempTr = temp.find_all("tr")[i]             # 소스코드 중 tr을 확인.
            row = {}
            column_idx = 0
            for j in range(0, 4):                       # 번호, 주민센터명, 대표전화번호, 주소 이 4개에 대해서 각각 합침.
                tempTd = tempTr.find_all("td")[j].text  
                row[column[column_idx]] = tempTd
                column_idx += 1
            df3 = df3.append(row,ignore_index=True)
        df4 = df4.append(df3)

# 다음 페이지(101번호)로 넘기기
    pagedown = driver.find_element(By.XPATH, '//*[@id="page_last_a"]') 
    pagedown.send_keys(Keys.ENTER)
    time.sleep(1)
df4

,번호,주민센터명,대표전화번호,주소
0,1,청운효자동 주민센터,02-2148-5002,서울특별시 종로구 자하문로 92
1,2,사직동 주민센터,02-2148-5033,서울특별시 종로구 사직로9길 1
2,3,종로1-4가동 주민센터,02-2148-5243,서울특별시 종로구 종로17길 8
3,4,삼청동 주민센터,02-2148-5062,서울특별시 종로구 삼청로 107
4,5,가회동 주민센터,02-2148-5213,서울특별시 종로구 북촌로 35
...,...,...,...,...
5,496,수민동 주민센터,051-550-6301,부산광역시 동래구 온천천로339번길 48
6,497,복산동 주민센터,051-550-6321,부산광역시 동래구 복천로 1
7,498,명륜동 주민센터,051-550-6361,부산광역시 동래구 명륜로207번길 26
8,499,온천1동 주민센터,051-550-6381,부산광역시 동래구 금강로 127


In [ ]:
df4 = df4.reset_index()
df4.head(1)

,index,번호,주민센터명,대표전화번호,주소
0,0,1,청운효자동 주민센터,02-2148-5002,서울특별시 종로구 자하문로 92


In [ ]:
df4 = df4.iloc[:,1:]
df4 = df4.iloc[:424]
df4

,번호,주민센터명,대표전화번호,주소
0,1,청운효자동 주민센터,02-2148-5002,서울특별시 종로구 자하문로 92
1,2,사직동 주민센터,02-2148-5033,서울특별시 종로구 사직로9길 1
2,3,종로1-4가동 주민센터,02-2148-5243,서울특별시 종로구 종로17길 8
3,4,삼청동 주민센터,02-2148-5062,서울특별시 종로구 삼청로 107
4,5,가회동 주민센터,02-2148-5213,서울특별시 종로구 북촌로 35
...,...,...,...,...
419,420,성내3동 주민센터,02-3425-7820,서울특별시 강동구 강동대로53길 76
420,421,천호1동 주민센터,02-3425-7670,서울특별시 강동구 구천면로42길 59
421,422,천호2동 주민센터,02-3425-7700,서울특별시 강동구 올림픽로 698
422,423,천호3동 주민센터,02-3425-7730,서울특별시 강동구 진황도로23길 7


In [ ]:
# df.to_csv('서울 주민센터.csv')

## 안됐던 90~100

In [ ]:
# 소스코드
page = driver.find_element(By.XPATH, '//*[@id="pageDiv"]/ul/li[12]/a')
page.send_keys(Keys.ENTER)
time.sleep(2)
html = driver.page_source
bsObject = BeautifulSoup(html, 'html.parser')
temp = bsObject.find_all("table")[0]
column = ["번호", "주민센터명", "대표전화번호", "주소"]

df10 = pd.DataFrame(columns = column)
for i in range(1, 11):
    tempTr = temp.find_all("tr")[i]
    row = {}
    column_idx = 0
    for j in range(0, 4):
        tempTd = tempTr.find_all("td")[j].text
        row[column[column_idx]] = tempTd
        column_idx += 1
    df10 = df10.append(row,ignore_index=True)
df10